# CBOW

In [14]:
import pandas as pd 

artigo_treino = pd.read_csv("../Aula 5 /treino.csv")
artigo_teste = pd.read_csv("../Aula 5 /teste.csv")

In [15]:
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download pt_core_news_sm

In [16]:
import spacy

nlp = spacy.load("pt_core_news_sm")

In [17]:
texto = "Adoro a cidade de Caldas novas"

doc = nlp(texto)

type(doc)

spacy.tokens.doc.Doc

In [18]:
doc.ents

(Adoro, Caldas novas)

In [19]:
doc[2].is_stop

False

In [20]:
doc[1].is_alpha # é alphanumerico

True

In [21]:
doc[1].is_stop #Verifica se é uma stopWord

True

In [22]:

def tratar_textos(doc): #tirar_stopwords e numeros 
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    if len(tokens_validos) > 2: #word2vec precisa no minimo uma frase com 3 palavras para o contexto
        return " ".join(tokens_validos)

In [23]:
texto = "Adoro 255255 cidade de Caldas Novas!"
doc = nlp(texto)
tratar_textos(doc)

'Adoro cidade Caldas'

In [24]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

textos_tratados = [tratar_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                         batch_size=100,
                                                         n_process= -1)]

In [25]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [26]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, # 0 para cbow e 1 skip gram 
               window= 2, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

In [27]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [28]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [29]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [30]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 

In [31]:
w2v_modelo.corpus_count

84466

In [32]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
        print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss

In [33]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples= w2v_modelo.corpus_count,
                 epochs= 30,
                 compute_loss= True,
                 callbacks=[callback()])

Loss após a época 0: 442060.40625
Loss após a época 1: 103368.09375
Loss após a época 2: 42868.875
Loss após a época 3: 25541.0
Loss após a época 4: 19117.6875
Loss após a época 5: 15363.1875
Loss após a época 6: 13024.125
Loss após a época 7: 11972.875
Loss após a época 8: 9449.5625
Loss após a época 9: 8579.4375
Loss após a época 10: 8727.0
Loss após a época 11: 7378.0
Loss após a época 12: 6894.5625
Loss após a época 13: 6612.6875
Loss após a época 14: 6311.8125
Loss após a época 15: 6485.5
Loss após a época 16: 5642.4375
Loss após a época 17: 5201.5625
Loss após a época 18: 5429.75
Loss após a época 19: 5055.375
Loss após a época 20: 5228.9375
Loss após a época 21: 4839.9375
Loss após a época 22: 4756.625
Loss após a época 23: 4443.6875
Loss após a época 24: 4409.1875
Loss após a época 25: 4279.8125
Loss após a época 26: 3766.375
Loss após a época 27: 3486.8125
Loss após a época 28: 3346.8125
Loss após a época 29: 3186.8125


(14584190, 16207260)

In [34]:
w2v_modelo.wv.most_similar("google")

[('facebook', 0.5761243104934692),
 ('abandono', 0.5467891693115234),
 ('natura', 0.46859440207481384),
 ('patriots', 0.4680592715740204),
 ('gramado', 0.46723243594169617),
 ('blur', 0.46623989939689636),
 ('vencedores', 0.46078771352767944),
 ('atenua', 0.4480825960636139),
 ('pausa', 0.4403047263622284),
 ('gravadora', 0.43990692496299744)]

# Skip Gram


from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, # 0 para cbow e 1 skip gram 
               window= 2, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

In [38]:
from gensim.models import Word2Vec

w2v_modelo_sg = Word2Vec(sg = 0, # 0 para cbow e 1 skip gram 
               window= 5, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

w2v_modelo_sg.build_vocab(lista_lista_tokens)



In [39]:
w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples= w2v_modelo.corpus_count,
                 epochs= 30,
                 compute_loss= True,
                 callbacks=[callback()])

Loss após a época 0: 425160.5625
Loss após a época 1: 89842.0625
Loss após a época 2: 32921.5
Loss após a época 3: 22614.75
Loss após a época 4: 16030.0625
Loss após a época 5: 12042.3125
Loss após a época 6: 10930.0625
Loss após a época 7: 9708.4375
Loss após a época 8: 8658.0625
Loss após a época 9: 8240.1875
Loss após a época 10: 7119.5625
Loss após a época 11: 6966.5625
Loss após a época 12: 6973.25
Loss após a época 13: 5846.875
Loss após a época 14: 5971.6875
Loss após a época 15: 5834.375
Loss após a época 16: 5155.5625
Loss após a época 17: 5388.625
Loss após a época 18: 4869.6875
Loss após a época 19: 5300.375
Loss após a época 20: 4814.625
Loss após a época 21: 4517.125
Loss após a época 22: 4448.0
Loss após a época 23: 4477.5
Loss após a época 24: 4150.0
Loss após a época 25: 4057.6875
Loss após a época 26: 3982.375
Loss após a época 27: 4524.5625
Loss após a época 28: 3934.5
Loss após a época 29: 3706.25


(14584037, 16207260)

In [40]:
w2v_modelo_sg.wv.most_similar("google")

[('apps', 0.4620926082134247),
 ('trato', 0.43152812123298645),
 ('feira', 0.4153140187263489),
 ('unifesp', 0.41097742319107056),
 ('britânica', 0.41091638803482056),
 ('nanini', 0.40695855021476746),
 ('influentes', 0.4050612449645996),
 ('estilista', 0.39860036969184875),
 ('acordo', 0.39123257994651794),
 ('tédio', 0.3904620110988617)]